In [1]:
from datetime import datetime, timezone
import glob

from climatology import compute_clim_mon
from xr_wrap import open_mfdataset_wrap

# function that generates list of input filenames

In [2]:
def gen_fnames_in(institute, model, experiment, rip, MIP_table, varname, grid):

    base_dir = '/glade/collections/cdg/data/CMIP6/CMIP'

    dir_in = '/'.join([base_dir, institute, model, experiment, rip, MIP_table, varname, grid, 'latest'])
    fname_pattern = '_'.join([varname, MIP_table, model, experiment, rip, grid, '*.nc'])
    fnames_in = glob.glob(dir_in+'/'+fname_pattern)
    fnames_in.sort()
    return fnames_in

# function that generates climatology

In [3]:
def gen_clim(fnames_in, yr_lo, yr_hi):

    time_name = 'time'

    # data_vars = 'minimal', to avoid introducing time dimension
    # to time-invariant fields when there are multiple files
    ds_in = open_mfdataset_wrap(fnames_in, chunks={time_name: 12}, data_vars='minimal', combine='by_coords')

    ds_clim = compute_clim_mon(ds_in, yr_lo, yr_hi)

    # change some global attributes that are not appropriate after postprocessing, document input files
    ds_clim.attrs['contact'] = 'klindsay@ucar.edu'
    ds_clim.attrs['creation_date'] = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")
    ds_clim.attrs['input_file_list'] = ' '.join(fnames_in)

    return ds_clim

# function that generates output filename

In [4]:
def gen_fname_out(dir_out, varname, MIP_table, model, experiment, rip, grid, yr_lo, yr_hi):

    yr_fname_string = yr_lo + '-' + yr_hi

    fname_out = '_'.join([varname, MIP_table, model, experiment, rip, grid, yr_fname_string, 'mon_clim.nc'])

    return '/'.join([dir_out, fname_out])

# call functions defined above with appropriately defined arguments

In [5]:
institute = 'NCAR'
model = 'CESM2'
experiment = 'historical'
rip = 'r7i1p1f1'
MIP_table = 'Omon'
grid = 'gn'

yr_lo = '1999'
yr_hi = '2000'

varname = 'fgco2'

dir_out = '/glade/scratch/klindsay/CESM2_CMIP6_ocnBgchem_tst'

fnames_in = gen_fnames_in(institute, model, experiment, rip, MIP_table, varname, grid)
print(fnames_in)

ds_clim = gen_clim(fnames_in, yr_lo, yr_hi)

fname_out = gen_fname_out(dir_out, varname, MIP_table, model, experiment, rip, grid, yr_lo, yr_hi)
print(fname_out)

ds_clim.to_netcdf(fname_out)

['/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_185001-189912.nc', '/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_190001-194912.nc', '/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_195001-199912.nc', '/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_200001-201412.nc']


/glade/work/klindsay/miniconda3/envs/CESM2_CMIP6_ocnBgchem/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'fgco2' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/glade/work/klindsay/miniconda3/envs/CESM2_CMIP6_ocnBgchem/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'fgco2' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


/glade/scratch/klindsay/CESM2_CMIP6_ocnBgchem_tst/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_1999-2000_mon_clim.nc


/glade/work/klindsay/miniconda3/envs/CESM2_CMIP6_ocnBgchem/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


# Correctness of climatology is checked with ferret script mon_clim_test_verify.jnl